In [1]:
import lightgbm as lgb
import sys
from os import listdir
import os.path
from os.path import isdir, isfile, join
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
def readCSVfromDirectory(dir):
    print(dir)
    filepaths = []
    for f in listdir(dir):
        path = join(dir, f)
        if isfile(path) and f.endswith(".csv"):
            filepaths.append(path)

    if len(filepaths) == 0:
        exit("Directory does not contain any csv files: %s" % dir)

    filepaths = np.sort(filepaths)
    print("Found %s files." % len(filepaths))

    return filepaths


In [58]:
#############################################
#MARK: - Read Files

SOURCE_DIR = "./data/"


SOURCE_DIR = os.path.abspath(SOURCE_DIR)
if not isdir(SOURCE_DIR):
    exit("That's not a directory!")
else:
    print(SOURCE_DIR)

elems = listdir(SOURCE_DIR)
print(elems)
directories = {}
for e in elems:
    path = join(SOURCE_DIR, e)
    
    if isdir(path):
        filepaths = readCSVfromDirectory(path)

        directories[e] = []
        for filepath in filepaths:
            # the files contain values in [V] and [A]
            # use a current Factor of 10e-3 if given in [mA]
            dataframe = pd.read_csv(filepath, header=1)
            # Felix ist schuld
            dataframe.columns = dataframe.columns.str.strip().str.replace('(', '').str.replace(')', '') 
            #create id field
            dataframe = dataframe.assign(id= lambda x: dataframe.index )
            #remove ms field
            dataframe = dataframe.drop(columns=['ms'])
            if 'mA' in dataframe:
                dataframe = dataframe.assign(A= lambda x: x.mA / 1000)
                dataframe = dataframe.drop(columns=['mA'])
                #print("Converted mA to A")
            directories[e].append(dataframe)
            

/Users/paul/DataScienceProjekt/data
['ohp_voll-single.psdata', 'laptop-single', 'ohp_voll-laptop', 'lampe-ohp_voll.psdata', 'ohp_halb-single', 'ohp_halb-single.psdata', 'ohp_voll-laptop-bosch-lampe.psdata', 'lampe-ohp_voll', 'ohp_voll-single', 'ohp_halb-laptop-lampe.psdata', 'ohp_halb-laptop.psdata', 'bosch-single', 'lampe-single', 'ohp_voll-laptop-bosch-lampe', 'ohp_halb-laptop', 'ohp_halb-laptop-lampe']
/Users/paul/DataScienceProjekt/data/laptop-single
Found 256 files.
/Users/paul/DataScienceProjekt/data/ohp_voll-laptop
Found 256 files.
/Users/paul/DataScienceProjekt/data/ohp_halb-single
Found 256 files.
/Users/paul/DataScienceProjekt/data/lampe-ohp_voll
Found 256 files.
/Users/paul/DataScienceProjekt/data/ohp_voll-single
Found 256 files.
/Users/paul/DataScienceProjekt/data/bosch-single
Found 256 files.
/Users/paul/DataScienceProjekt/data/lampe-single
Found 256 files.
/Users/paul/DataScienceProjekt/data/ohp_voll-laptop-bosch-lampe
Found 256 files.
/Users/paul/DataScienceProjekt/data/

In [60]:
for (id, dfs) in directories.items():
    print(id, len(dfs))
    print(dfs[0].head(4))
    print(dfs[0].describe())
    print('\n\n')

laptop-single 256
          V         A  id
0 -8.224738  0.018067   0
1 -8.224738  0.036012   1
2 -3.753777  0.018067   2
3  0.732444  0.018067   3
                 V            A           id
count  2445.000000  2445.000000  2445.000000
mean     38.888648     0.021008  1222.000000
std     222.962687     0.338291   705.955027
min    -321.848200    -1.293191     0.000000
25%    -178.472200    -0.017823   611.000000
50%      81.362350     0.000122  1222.000000
75%     251.609800     0.018067  1833.000000
max     314.340600     1.419172  2444.000000



ohp_voll-laptop 256
           V         A  id
0 -12.710960 -1.078524   0
1  -8.224738 -1.078524   1
2  -3.753777 -1.078524   2
3  -8.224738 -1.078524   3
                 V            A           id
count  2445.000000  2445.000000  2445.000000
mean     38.879487     0.443960  1222.000000
std     221.681810     2.560982   705.955027
min    -317.362000    -5.035249     0.000000
25%    -174.001300    -1.887875   611.000000
50%      81.362350 

#### TODO

- Zeit diskretisieren 0ms = 0 und 20ms = 2445 (neue spalte -> id des datensatzes) done
- mA zu A umrechnen (falls vorhanden) done
- geht multilabel mit gradient boosting? jaein, vielleicht
- in X und Y aufteilen 
- attacke!


In [114]:
features = []

for label, data in directories.items():
    # pro gerät alle daten in einen dataframe
    res = pd.concat(data, sort=False)
    #zu der tabelle eine spalte mit label hinzufügen
    res = res.assign(labels= lambda x: label)
    
    features.append(res)

# dataframes zu einem kombinieren
features = pd.concat(features, sort=False)

X = features.drop(columns=['labels'])
Y = features['labels']


Y = pd.get_dummies(Y)
Y = Y.assign(labels= lambda x: 0 + x['lampe-ohp_voll'] + x['lampe-single'] * 2 + x['laptop-single'] * 3 + x['ohp_halb-laptop']* 4 + x['ohp_halb-laptop-lampe'] * 5 + x['ohp_halb-single'] * 6 + x['ohp_voll-laptop'] * 7 + x['ohp_voll-laptop-bosch-lampe'] * 8 + x['ohp_voll-single'] *9)
Y = Y['labels']
print(Y.head(4))

0    3
1    3
2    3
3    3
Name: labels, dtype: uint8


In [115]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state=42)

In [116]:
train_data = lgb.Dataset(X_train, label=Y_train)

test_data = lgb.Dataset(X_test, label=Y_test)

In [ ]:
param = {'num_leaves':31, 'num_trees':100, 'objective':'multiclass', 'num_class': 10}
num_round = 10

bst = lgb.train(param, train_data, num_round, valid_sets=[test_data])

/Users/paul/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:102: UserWarning: Found `num_trees` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[1]	valid_0's multi_logloss: 2.0965
[2]	valid_0's multi_logloss: 1.95911
[3]	valid_0's multi_logloss: 1.85223
[4]	valid_0's multi_logloss: 1.76508
[5]	valid_0's multi_logloss: 1.69055
[6]	valid_0's multi_logloss: 1.62431
[7]	valid_0's multi_logloss: 1.56658
[8]	valid_0's multi_logloss: 1.51757
[9]	valid_0's multi_logloss: 1.47059
[10]	valid_0's multi_logloss: 1.42865
[11]	valid_0's multi_logloss: 1.39026
[12]	valid_0's multi_logloss: 1.35616
[13]	valid_0's multi_logloss: 1.32462
[14]	valid_0's multi_logloss: 1.29463
[15]	valid_0's multi_logloss: 1.26692
[16]	valid_0's multi_logloss: 1.2412
[17]	valid_0's multi_logloss: 1.21683
[18]	valid_0's multi_logloss: 1.19435
[19]	valid_0's multi_logloss: 1.17399
[20]	valid_0's multi_logloss: 1.15454
[21]	valid_0's multi_logloss: 1.13504
[22]	valid_0's multi_logloss: 1.11694
[23]	valid_0's multi_logloss: 1.10107
[24]	valid_0's multi_logloss: 1.08604
[25]	valid_0's multi_logloss: 1.07002
[26]	valid_0's multi_logloss: 1.05608
[27]	valid_0's multi_lo